# **Challenge 9 : Model Deployment**

This Colab notebook explores deploying a machine translation model using Gradio. It depends on a pre-trained english to french translation model from Challenge 7

# *Step 1 : Installing Gradio*

In [8]:
!pip install gradio

In [9]:
# Mounting my drive to access necessary files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#*Step 2 : Setting up the Gradio app*

In [10]:
#Importing the required libraries
import gradio as gr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

Loading the necessary files

In [11]:
''' Loading the necessary files'''

# Loading the saved model
model = load_model('/content/drive/MyDrive/Colab_Notebooks/ZAKA_AIC/Module7/french_to_english_model.keras')

# Loading the tokenizers
with open('/content/drive/MyDrive/Colab_Notebooks/ZAKA_AIC/Module7/english_tokenizer.pkl', 'rb') as f:
    english_tokenizer = pickle.load(f)
with open('/content/drive/MyDrive/Colab_Notebooks/ZAKA_AIC/Module7/french_tokenizer.pkl', 'rb') as f:
    french_tokenizer = pickle.load(f)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 16 variables whereas the saved optimizer has 30 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Adding the translation function

In [13]:
# Defining the translation function
def translate_sentence(input_sentence):
    input_sequence = english_tokenizer.texts_to_sequences([input_sentence])
    input_padded = pad_sequences(input_sequence, maxlen=15, padding='post')
    predictions = model.predict(input_padded)
    predicted_sequence = predictions[0].argmax(axis=-1)
    reverse_french_word_index = {v: k for k, v in french_tokenizer.word_index.items()}
    translated_sentence = ' '.join([reverse_french_word_index.get(idx, '') for idx in predicted_sequence if idx > 0])
    return translated_sentence

Gradio Interface

In [14]:
''' Defining and Customizing the Gradio Interfase'''

# Function to clear the input
def clear_input():
    return ""

# Customizing the Gradio interface
title = "🌐 French to English Translator"
description = """
Enter a sentence in English, and this application will translate it into French for you!
"""

# Defining the Gradio Interface
with gr.Blocks() as interface:
    # Add the GIF
    gr.HTML("""
    <div style="text-align: center;">
        <img src="https://media.giphy.com/media/IEIVNS0WEgbPZlX4Md/giphy.gif"
        style="display: block; margin: 0 auto; width: 150px; height: 150px; margin-bottom: 20px;">
    </div>
    """)

    # Adding Title and Description
    gr.HTML(f"<h1 style='color: #d81b60; text-align: center;'>{title}</h1>")
    gr.HTML(f"<p style='text-align: center; color: gray;'>{description}</p>")

    # Centering Input and Output Form
    with gr.Row():
        with gr.Column():
            input_box = gr.Textbox(
                label="Input Sentence",
                placeholder="Type your sentence in English...",
                lines=2,
                elem_id="input_box"
            )
        with gr.Column():
            output_box = gr.Textbox(
                label="Translated Sentence",
                interactive=False,
                elem_id="output_box"
            )

    # Adding Translate and Clear Buttons
    with gr.Row():
        translate_button = gr.Button("Translate")
        clear_button = gr.Button("Clear")

    # Button Actions
    translate_button.click(
        translate_sentence,
        inputs=input_box,
        outputs=output_box
    )
    clear_button.click(
        clear_input,
        inputs=None,
        outputs=input_box
    )

    # Adding Custom Styles
    gr.HTML("""
    <style>
        body {
            background-color: #f8f9fa;
        }
        #input_box, #output_box {
            border-radius: 10px;
            border: 2px solid #d81b60;
        }
        #translate_button, #clear_button {
            background-color: #d81b60;
            border: none;
            color: white;
            padding: 10px 20px;
            font-size: 16px;
            border-radius: 10px;
            cursor: pointer;
        }
        #translate_button:hover, #clear_button:hover {
            background-color: #b71550;
        }
    </style>
    """)

#**Step 3: Launching and Accessing the App**

In [15]:
# Launching the Gradio interface
interface.launch(share=True)  # `share=True` creates a public URL

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://862aa6eda63994df9f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
